In [27]:
import pandas as pd
from bs4 import BeautifulSoup
import time
import requests
from selenium.webdriver.common.by import By
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.actions.wheel_input import ScrollOrigin
import openpyxl

In [73]:
class crawl:
    titles = []
    prices = []
    prices_org = []
    off = []
    rating = []
    links = []
    caption = []
    priced = []
    
    def source(self , link , sheet_name):
        self.titles = []
        self.prices = []
        self.prices_org = []
        self.off = []
        self.rating = []
        self.links = []
        self.caption = []
        # Open browser
        driver_path = r"E:\chromedriver-win64\chromedriver.exe"
        service = Service(executable_path=driver_path)
        driver = Chrome(service=service)
        driver.get(link)
        # Scroll to load content
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            # Scroll down
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            # Loading
            time.sleep(2)
            # New height
            new_height = driver.execute_script("return document.body.scrollHeight")
            # Checking if we have reached the end of the page
            if new_height == last_height:
                break
            last_height = new_height
        # Save source
        source = driver.page_source
        driver.quit()
        soup = BeautifulSoup(source ,'html.parser')
        # Collect product links
        products = soup.find_all("a" , {"class" :"styles__ProductTitle-sc-wary-5 huiqyt product-title block"})
        # Get the source of each product
        for product in products:
            PATH = "https://www.headout.com" + product['href']
            self.links.append(PATH)
            page = requests.get(PATH)
            page_source = BeautifulSoup(page.text ,'html.parser')
            self.crawling(page_source)
        self.save(sheet_name)
        
    def crawling(self , source):
        try :
            self.titles.append(source.find('h1' , {'data-qa-marker' : 'product-name'}).text)
        except :
            self.titles.append(None)
        
        try :
            self.caption.append(source.find('div' , {'class' : 'content-html notranslate'}).find('p').text)
        except :
            self.caption.append(None)
            
        try :
            self.rating.append(source.find('span' , {'class' : 'product-rating-value rating'}).text + f"({source.find('div' , {'class' : 'product-rating-text'}).find('span').text})")
        except :
            self.rating.append(None)
                
        try :
            self.prices.append(source.find('div' , {'class' : 'tourBasicPriceWrapper__FinalPriceText-sc-1ptsmk1-4 cNycAP discounted'}).text.replace('\xa0',''))
        except AttributeError:                
            self.prices.append(source.find('div' , {'class' : 'tourBasicPriceWrapper__FinalPriceText-sc-1ptsmk1-4 cNycAP no-discount'}).text.replace('\xa0',''))
        except :
            self.prices.append(None)
                
        try :
            self.off.append(source.find('span' , {'class' : 'notranslate discount-percentage'}).text.replace('\xa0',''))
        except :
            self.off.append(None)
                
        try :
            self.prices_org.append(source.find('div' , {'class' : 'tourBasicPriceWrapper__OriginalPriceText-sc-1ptsmk1-5 gPIjQZ'}).text.replace('\xa0',''))
        except :
            self.prices_org.append(None)

    def save(self , sheet_name):
        # Save into dictionary
        dict = {}
        dict['titles'] = self.titles
        dict['rating'] = self.rating
        dict['links'] = self.links 
        dict['caption'] = self.caption
        dict['orginal prices'] = self.prices_org 
        dict['off'] = self.off 
        dict['prices'] = self.prices 
        # Save the previous price in a separate list
        try :
            workbook = openpyxl.load_workbook('Headout.xlsx')
            # The list of sheets in the file
            sheet_names = workbook.sheetnames
            # Checking the existence of sheets
            sheet_name_to_check = sheet_name
            if sheet_name_to_check in sheet_names :
                dfb = pd.read_excel('Headout.xlsx', sheet_name=sheet_name)
                dict['previous_prices'] = dfb['prices']
                for i in range(len(self.prices)):
                        price1 = float(self.prices[i].replace('AED', '').strip())
                        price2 = float(dfb['prices'][i].replace('AED', '').strip())
                        self.priced.append(f"{(price1 - price2)}AED")
                dict['price_diffrence'] = self.priced
        except :
            pass
        # Save into dataframe
        df = pd.DataFrame(dict)
        try:
            with pd.ExcelWriter('Headout.xlsx', mode='a', if_sheet_exists='replace') as writer:
                df.to_excel(writer , sheet_name=sheet_name , index=False)
        except FileNotFoundError:
            df.to_excel("Headout.xlsx" , sheet_name=sheet_name, index=False)
        

In [75]:
ob1 = crawl()

In [ ]:
ob1.source("https://www.headout.com/theme-parks-dubai-sc-1001~6/?limit=100&offset=0" , "Theme Parks")

In [ ]:
ob1.source("https://www.headout.com/museums-dubai-sc-1002~6/?limit=100&offset=0" , "Museums")

In [ ]:
ob1.source("https://www.headout.com/zoos-dubai-sc-1003~6/?limit=100&offset=0" , "Zoos")

In [77]:
ob1.source("https://www.headout.com/parks-dubai-sc-1004~6/?limit=100&offset=0" , "Parks")

In [ ]:
ob1.source("https://www.headout.com/water-parks-dubai-sc-1005~6/?limit=100&offset=0" , "Water Parks")

In [ ]:
ob1.source("https://www.headout.com/religious-sites-dubai-sc-1006~6/?limit=100&offset=0" , "Religious Sites")

In [ ]:
ob1.source("https://www.headout.com/landmarks-dubai-sc-1007~6/?limit=100&offset=0" , "Landmarks")

In [ ]:
ob1.source("https://www.headout.com/city-cards-dubai-sc-1008~6/?limit=100&offset=0" , "City Cards")

In [ ]:
ob1.source("https://www.headout.com/observation-decks-dubai-sc-1098~6/?limit=100&offset=0" , "Observation Decks")

In [ ]:
ob1.source("https://www.headout.com/tours-dubai-ca-2~6/?limit=100&offset=0" , "Tours")

In [ ]:
ob1.source("https://www.headout.com/transportation-dubai-ca-3~6/?limit=100&offset=0" , "Transportation")

In [ ]:
ob1.source("https://www.headout.com/travel-services-dubai-ca-4~6/?limit=100&offset=0" , "Travel Services")

In [ ]:
ob1.source("https://www.headout.com/cruises-dubai-ca-18~6/?limit=100&offset=0" , "Cruises")

In [ ]:
ob1.source("https://www.headout.com/food-drink-dubai-ca-5~6/?limit=100&offset=0" , "Food & Drink")

In [ ]:
ob1.source("https://www.headout.com/entertainment-dubai-ca-7~6/?limit=100&offset=0" , "Entertainment shows")

In [ ]:
ob1.source("https://www.headout.com/adventure-dubai-ca-8~6/?limit=100&offset=0" , "Adventure")

In [ ]:
ob1.source("https://www.headout.com/aerial-sightseeing-dubai-ca-9~6/limit=100&offset=0" , "Aerial Sightseeing")

In [ ]:
ob1.source("https://www.headout.com/water-sports-dubai-ca-10~6/?limit=100&offset=0" , "Water Sports")

In [ ]:
ob1.source("https://www.headout.com/specials-dubai-ca-14~6/?limit=100&offset=0" , "Specials")

In [ ]:
ob1.source("https://www.headout.com/sports-dubai-ca-19~6/?limit=100&offset=0" , "Sports")